In [1]:
import salabim as sim

## With Queue

In [5]:
class CustomerGenerator(sim.Component):
    def process(self):
        while True:
            Customer()
            self.hold(sim.Uniform(5, 15).sample())


class Customer(sim.Component):
    def process(self):
        if len(waitingline) >= 5:
            env.number_balked += 1
            env.print_trace("", "", "balked")
            print(env.now(), "balked",self.name())
            self.cancel()
        self.enter(waitingline)
        for clerk in clerks:
            if clerk.ispassive():
                clerk.activate()
                break #activate only 1 clerk
        self.hold(50) # if not served within this time -> renege
        if self in waitingline:
            self.leave(waitingline)
            env.number_reneged += 1
            env.print_trace("", "", "reneged")
        else:
            self.passivate() # waiting for service to be completed


class Clerk(sim.Component):
    def process(self):
        while True:
            while len(waitingline) == 0:
                self.passivate()
            self.customer = waitingline.pop()
            self.customer.activate() # get the customer out of its hold
            self.hold(30)
            self.customer.activate() # signal the customer that all is done



env = sim.Environment(trace=True)

CustomerGenerator()
env.number_balked = 0
env.number_reneged = 0
clerks = [Clerk() for _ in range(3)]
waitingline = sim.Queue('waitingline')


env.run(duration=3000) # till can also be used
waitingline.length.print_histogram(30,0,1)
waitingline.length_of_stay.print_histogram(30,0,10)
print(f'number balked: {env.number_balked}')
print(f'number reneged: {env.number_reneged}')

# waitingline.print_histograms()
# waitingline.print_info()
# print()
# waitingline.print_statistics()

## With Store

In [ ]:

class CustomerGenerator(sim.Component):
    def process(self):
        while True:
            Customer().enter(waiting_room)
            self.hold(sim.Uniform(5, 15).sample())


class Customer(sim.Component):
    ...


class Clerk(sim.Component):
    def process(self):
        while True:
            customer = self.from_store(waiting_room)
            self.hold(30)
            

env = sim.Environment(trace=True)

CustomerGenerator()
for _ in range(3):
    Clerk()
waiting_room = sim.Store('waiting_room')


env.run(till=5000)
waiting_room.print_statistics()
waiting_room.print_info()


## With Resources

In [ ]:
class CustomerGenerator(sim.Component):
    def process(self):
        while True:
            Customer()
            self.hold(sim.Uniform(5, 15).sample())


class Customer(sim.Component):
    def process(self):
        self.request(clerks)
        self.hold(30)
        self.release()
    


env = sim.Environment(trace=True)

CustomerGenerator()
for _ in range(3):
    Clerk()
clerks = sim.Resource("clerks", capacity=3)

env.run(till=5000)
clerks.print_statistics()
clerks.print_info()
